# Machine Learning on Distributed Dask with SageMaker and Fargate

This notebook will demonstrate how to perform Machine Learning on Distributed Dask using SageMaker and Fargate.  We will demo how to connect to distributed dask fargate cluster, scale out dask worker nodes, perform EDA work on public newyork cab trip data sets. Then, we demonstrate how you can run regression algorithms and hyperparameters optimization on distributed dask cluster. Next, we will demonstrate how you can monitor the operational metrics of Dask Cluster that will be fronted by Network Load Balancer for accessing the Dask Cluster Status UI from internet. Finally, we will close with how to build your own python script container and run against the dask fargate cluster.  This notebook was inspired by customer use case where they were running dask on local computer for building regression models.   

## Connect to Dask Fargate Cluster.  You need to provision this cluster following the instructions from here https://github.com/rvvittal/aws-dask-sm-fargate

In [1]:
from dask.distributed import Client

#enable this client for local device testing
client = Client('localhost:8786')

#enable this client for sagemaker notebook testing
#client = Client('dask-scheduler.local-dask:8786')

/Users/rvvittal/opt/anaconda3/lib/python3.7/site-packages/distributed/client.py:1079: VersionMismatchWarning: Mismatched versions found

blosc
+------------------------+---------+
|                        | version |
+------------------------+---------+
| client                 | None    |
| scheduler              | 1.7.0   |
| tcp://172.20.0.2:39363 | 1.7.0   |
| tcp://172.20.0.3:45397 | 1.7.0   |
+------------------------+---------+

python
+------------------------+----------------+
|                        | version        |
+------------------------+----------------+
| client                 | 3.7.6.final.0  |
| scheduler              | 3.6.10.final.0 |
| tcp://172.20.0.2:39363 | 3.6.10.final.0 |
| tcp://172.20.0.3:45397 | 3.6.10.final.0 |
+------------------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


## Scale out the number of dask workers as needed for your data science work

In [ ]:
#enable this  when cluster is running on Fargate to scale out your cluster. 
#!aws ecs update-service --service Dask-Workers --desired-count 10 --cluster Fargate-Dask-Cluster

## Restart the client after scale out operation

In [ ]:
client.restart()

## Introduction to Dask DataFrame
A Dask DataFrame is a large parallel DataFrame composed of many smaller Pandas DataFrames, split along the index. These Pandas DataFrames may live on disk for larger-than-memory computing on a single machine, or on many different machines in a cluster. One Dask DataFrame operation triggers many operations on the constituent Pandas DataFrames. For more details, review this page: https://docs.dask.org/en/latest/dataframe.html 



In [2]:
import s3fs
import dask.dataframe as dd
import boto3
import dask.distributed

## Using Dask for EDA on NewYork Taxi Trip datasets

In [3]:
df = dd.read_csv(
    's3://nyc-tlc/trip data/green_tripdata_2018-02.csv', storage_options={'anon': True}
)

In [4]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2018-02-01 00:39:38,2018-02-01 00:39:41,N,5,97,65,1,0.00,20.0,0.0,0.0,3.00,0.0,NaN,0.0,23.00,1,2
1,2,2018-02-01 00:58:28,2018-02-01 01:05:35,N,1,256,80,5,1.60,7.5,0.5,0.5,0.88,0.0,NaN,0.3,9.68,1,1
2,2,2018-02-01 00:56:05,2018-02-01 01:18:54,N,1,25,95,1,9.60,28.5,0.5,0.5,5.96,0.0,NaN,0.3,35.76,1,1
3,2,2018-02-01 00:12:40,2018-02-01 00:15:50,N,1,61,61,1,0.73,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,2,1
4,2,2018-02-01 00:45:18,2018-02-01 00:51:56,N,1,65,17,2,1.87,8.0,0.5,0.5,0.00,0.0,NaN,0.3,9.30,2,1


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [ ]:
# load and count number of rows
len(df)

In [ ]:
df.dtypes

## Persist  collections into memory
Calls to Client.compute or Client.persist submit task graphs to the cluster and return Future objects that point to particular output tasks. Compute returns a single future per input, persist returns a copy of the collection with each block or partition replaced by a single future. In short, use persist to keep full collection on the cluster and use compute when you want a small result as a single future.


In [ ]:
df_persisted = client.persist(df)
print(df_persisted.head())

## Compute the mean trip distance grouped by the number of passengers

In [ ]:
grouped_df = df.groupby(df_persisted.passenger_count).trip_distance.mean().compute()
print(grouped_df)

## Compute Max trip distance

In [ ]:
max_trip_dist = df_persisted.trip_distance.max().compute()
print(max_trip_dist)

## Count the total trip distance and count for each vendor

In [ ]:
%%time
df.groupby('VendorID').agg({'passenger_count':'count', 'trip_distance': 'sum'}).astype(int).reset_index()\
.rename(columns={'passenger_count':'Trip Count'}).compute()

## Count Missing Values for Each Feature

In [ ]:
df.isna().sum().compute()

##  Visual EDA  

In [ ]:
##Selecting top 10 rides based on fare amount
most_paid_rides_dask = df[['PULocationID', 'fare_amount']].nlargest(10, "fare_amount")

In [ ]:
##Visualizing most paid rides through Barplot
import matplotlib.pyplot as plt
most_paid_rides_dask.set_index('PULocationID',sorted=True).compute().plot(kind='barh',stacked=False, figsize=[10,8], legend=True)
#######
plt.title('Most Paid Rides')
plt.xlabel('Fare Amount')
plt.ylabel('PU LocationID')
plt.show()


In [ ]:
##Visualizing trip distance through Barplot
import matplotlib.pyplot as plt
most_paid_rides_dask2 = df[['trip_distance', 'fare_amount']].nlargest(10, "trip_distance")
most_paid_rides_dask2.set_index('trip_distance',sorted=True).compute().plot(kind='bar', colormap='PiYG', stacked=False, figsize=[10,8], legend=True)
#######
plt.title('Fares by Distance')
plt.xlabel('Trip Distance')
plt.ylabel('Fare Amount')
plt.show()

## Regression modeling with Scikit Learn

In [ ]:
dfl = dd.read_csv(
    's3://nyc-tlc/trip data/green_tripdata_2018-02.csv', storage_options={'anon': True},
    parse_dates=['lpep_pickup_datetime', 'lpep_dropoff_datetime'],
).sample(frac=0.1, replace=True)

In [ ]:
dfl['trip_duration'] = dfl['lpep_dropoff_datetime'] - dfl['lpep_pickup_datetime']

In [ ]:
import numpy as np
dfl['trip_duration'] = dfl['trip_duration']/np.timedelta64(1,'D')

In [ ]:
dfl['trip_duration'] = dfl['trip_duration'] * 24

In [ ]:
dfl['trip_duration']

In [ ]:
dfl.head()

In [ ]:
dfl = dfl.fillna(value=0)

In [ ]:
dfl = dd.get_dummies(dfl.categorize()).compute()

In [ ]:
dfl.head()

In [ ]:
x = dfl[['VendorID','RatecodeID','PULocationID','DOLocationID','passenger_count','trip_distance','fare_amount','total_amount']]

In [ ]:
y = dfl['trip_duration']

In [ ]:
from dask_ml.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
training_x = X_train.values
training_y = y_train.values

In [ ]:
testing_x = X_test.values
testing_y = y_test.values

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [ ]:
def rmse(preds, actuals):
    error = mean_squared_error(actuals, preds)
    rmse = np.sqrt(error)
    print(rmse)

In [ ]:
from dask_ml.linear_model import LinearRegression
lr = LinearRegression(random_state=1, n_jobs=-1, fit_intercept=True)
lr.fit(training_x,training_y)

In [ ]:
import joblib
from dask_ml.linear_model import LinearRegression

with joblib.parallel_backend('dask'):
    lr = LinearRegression(random_state=1, fit_intercept=False)
    lr.fit(training_x,training_y)

## Linear Regression with Dask distributed machine learning

In [ ]:
from dask_glm.datasets import make_regression
X, y = make_regression(n_samples=200000, n_features=100, n_informative=5, chunksize=10000)
X


In [ ]:
import dask
X, y = dask.persist(X, y)

In [ ]:
import dask_glm.algorithms

b = dask_glm.algorithms.admm(X, y, max_iter=5)

In [ ]:
b = dask_glm.algorithms.proximal_grad(X, y, max_iter=5)

In [ ]:
import dask_glm.families
import dask_glm.regularizers

family = dask_glm.families.Poisson()
regularizer = dask_glm.regularizers.ElasticNet()

b = dask_glm.algorithms.proximal_grad(
    X, y,
    max_iter=5,
    family=family,
    regularizer=regularizer,
)


## Hyperparameter Optimization with Dask distributed machine learning

Scikit-learn uses joblib for single-machine parallelism. This lets you train most estimators (anything that accepts an n_jobs parameter) using all the cores of your laptop or workstation.Alternatively, Scikit-Learn can use Dask for parallelism. This lets you train those estimators using all the cores of your cluster without significantly changing your code.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd


In [ ]:
X, y = make_classification(n_samples=1000, random_state=0)
X[:5]

In [ ]:
y[:5]

In [ ]:
param_grid = {"C": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
              "kernel": ['rbf', 'poly', 'sigmoid'],
              "shrinking": [True, False]}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True),
                           param_grid=param_grid,
                           return_train_score=False,
                           iid=True,
                           cv=3,
                           n_jobs=-1)


In [ ]:
grid_search.fit(X, y)

In [ ]:
import joblib

with joblib.parallel_backend('dask'):
    grid_search.fit(X, y)


In [ ]:
pd.DataFrame(grid_search.cv_results_).head()

## Run your python script container for your machine learning work.  

Make sure to follow the steps in github repo for building/deploying this container before running this step. Change {YOURACCOUNTID} to your account id

In [ ]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin {YOURACCOUNTID}.dkr.ecr.us-west-2.amazonaws.com

In [ ]:
!docker run -e s3url='s3://nyc-tlc/trip data/green_tripdata_2018-02.csv' -e schurl='tcp://Dask-Scheduler.local-dask:8786' {YOURACCOUNTID}.dkr.ecr.us-west-2.amazonaws.com/daskclientapp:latest

## Scale in the Fargate cluster worker nodes after all work is done

In [ ]:
!aws ecs update-service --service Dask-Workers --desired-count 1 --cluster Fargate-Dask-Cluster